In [34]:
!pip install scipy
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Zadanie 1

In [35]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go
import statsmodels.formula.api as smf

In [36]:
data = pd.read_csv("wyniki.csv")

In [37]:
data

,plec,ocena_koncowa
0,F,6
1,F,6
2,F,10
3,F,15
4,F,10
...,...,...
352,M,9
353,M,16
354,M,7
355,M,10


In [38]:
significance_level = 0.05

In [39]:
male_grades = data[data['plec'] == 'M'].reset_index()
female_grades = data[data['plec'] == 'F'].reset_index()
male_count = male_grades.shape[0]
female_count = female_grades.shape[0]

print(f'Female count : {female_count}    Male count : {male_count}')

Female count : 185    Male count : 172


In [40]:
data_for_t = pd.DataFrame(data={'sex': ['M', 'F'], 
                                'mean': [male_grades['ocena_koncowa'].mean(), female_grades['ocena_koncowa'].mean()],
                                'std': [male_grades['ocena_koncowa'].std(), female_grades['ocena_koncowa'].std()],
                                'nobs': [male_grades['ocena_koncowa'].count(), female_grades['ocena_koncowa'].count()] })

In [41]:
test_t = stats.ttest_ind_from_stats(data_for_t.iloc[0][1], data_for_t.iloc[0][2], data_for_t.iloc[0][3], data_for_t.iloc[1][1], data_for_t.iloc[1][2], data_for_t.iloc[1][3])

In [42]:
print(data_for_t)

  sex       mean       std  nobs
0   M  11.866279  3.258748   172
1   F  11.205405  3.174452   185


In [43]:
print(f'T : {test_t[0]}    P : {test_t[1]}')
print(f'Czy można odrzucić hipotezę zerową? {significance_level <= test_t[1]}')


T : 1.9404771273257526    P : 0.053113174845791505
Czy można odrzucić hipotezę zerową? True


## Zadanie 2


In [44]:
income = pd.read_csv("ZyskiFirmyX.csv")
income.head()

,Rok,Zysk
0,1821,1645
1,1822,658
2,1823,1926
3,1824,865
4,1825,764


In [45]:
df_incomes_histogram = income.copy()
df_incomes_histogram["Color"] = np.where(df_incomes_histogram["Zysk"] < 0, 'red', 'green')

fig = go.Figure()
fig.add_trace(
    go.Bar(
        name='Income per Year',
        x=df_incomes_histogram['Rok'],
        y=df_incomes_histogram['Zysk'],
        marker_color=df_incomes_histogram['Color']
    )
)
fig.update_layout(
    barmode='stack',
    xaxis=dict(
        dtick = 10
    ),
    title='Income per Year'
)
fig.show()

In [46]:
model = smf.ols(formula="Zysk ~ Rok", data=income).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Zysk   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.09357
Date:                Sat, 28 Jan 2023   Prob (F-statistic):              0.760
Time:                        09:28:48   Log-Likelihood:                -1521.3
No. Observations:                 200   AIC:                             3047.
Df Residuals:                     198   BIC:                             3053.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1373.8795   1150.888      1.194      0.2

In [47]:
df_incomes_histogram["fitted"] = model.fittedvalues
slope, intercept, r_value, p_value, std_err = stats.linregress(df_incomes_histogram["Rok"], df_incomes_histogram["fitted"])

In [48]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        name='Income per Year',
        x=df_incomes_histogram['Rok'],
        y=df_incomes_histogram['Zysk'],
        marker_color=df_incomes_histogram['Color']
    )
)
fig.add_trace(
    go.Scatter(
        x=df_incomes_histogram["Rok"], 
        y=df_incomes_histogram["fitted"], 
        name="Fitted Regression Line"
    )
)
fig.add_annotation(dict(
    font=dict(
        color='blue', size=15
    ),
    x=0.005,
    y=-0.23,
    text=f"Regression line formula: y = {round(slope, 3)}x + {round(intercept, 2)}",
    textangle=0,
    xanchor='left',
    xref="paper",
    yref="paper"
))

fig.update_layout(
    barmode='stack',
    title='Income per Year',
    xaxis=dict(
        dtick = 10
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(l=20, r=20, t=20, b=60)
)



fig.show()

## Zadanie 3

In [49]:
import seaborn as sns

In [50]:
df_spending_scores = pd.read_csv("spendingscores.csv")
df_spending_scores['Gender'] = df_spending_scores['Gender'].replace({
    "Male" : 0,
    "Female" : 1
})
df_spending_scores

,Gender,Age,AnnualIncome,SpendingScore
0,0,19,15,39
1,0,21,15,81
2,1,20,16,6
3,1,23,16,77
4,1,31,17,40
...,...,...,...,...
195,1,35,120,79
196,1,45,126,28
197,0,32,126,74
198,0,32,137,18


In [51]:
model_spending_scores = smf.ols(formula="SpendingScore ~ Gender + Age + AnnualIncome", data=df_spending_scores).fit()
print(model_spending_scores.summary())

                            OLS Regression Results                            
Dep. Variable:          SpendingScore   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     7.960
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           4.91e-05
Time:                        09:28:48   Log-Likelihood:                -922.05
No. Observations:                 200   AIC:                             1852.
Df Residuals:                     196   BIC:                             1865.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       71.9168      7.023     10.240   

In [52]:
print("Model P Values:", model_spending_scores.pvalues.values)
print("Model Std Errs:", model_spending_scores.bse.values)
print("Model Coef:", model_spending_scores.params.values)

Model P Values: [5.50097367e-20 5.67117187e-01 3.05517724e-06 9.05093933e-01]
Model Std Errs: [7.02279453 3.51182541 0.12491583 0.06641961]
Model Coef: [ 7.19167998e+01  2.01323375e+00 -6.00371040e-01  7.92939510e-03]


In [53]:
corr = df_spending_scores.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

<ipython-input-53-68f63a797533>:2: FutureWarning:

this method is deprecated in favour of `Styler.format(precision=..)`



,Gender,Age,AnnualIncome,SpendingScore
Gender,1.000,-0.061,-0.056,0.058
Age,-0.061,1.000,-0.012,-0.327
AnnualIncome,-0.056,-0.012,1.000,0.010
SpendingScore,0.058,-0.327,0.010,1.000


In [54]:
import plotly.express as px
px.imshow(df_spending_scores.corr(), color_continuous_scale='Agsunset', title="Correlation heatmap of spendingscores dataframe")

In [55]:
df_spending_scores_without_AnnualIncome = df_spending_scores.drop(columns=['AnnualIncome'])

model_spending_scores = smf.ols(formula="SpendingScore ~ Gender + Age", data=df_spending_scores_without_AnnualIncome).fit()
print(model_spending_scores.summary())

corr = df_spending_scores_without_AnnualIncome.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

                            OLS Regression Results                            
Dep. Variable:          SpendingScore   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     11.99
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           1.22e-05
Time:                        09:28:48   Log-Likelihood:                -922.05
No. Observations:                 200   AIC:                             1850.
Df Residuals:                     197   BIC:                             1860.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.4197      5.605     12.920      0.0

<ipython-input-55-c08c0efec2d7>:7: FutureWarning:

this method is deprecated in favour of `Styler.format(precision=..)`



,Gender,Age,SpendingScore
Gender,1.000,-0.061,0.058
Age,-0.061,1.000,-0.327
SpendingScore,0.058,-0.327,1.000


In [56]:
print("Model P Values:", model_spending_scores.pvalues.values)
print("Model Std Errs:", model_spending_scores.bse.values)
print("Model Coef:", model_spending_scores.params.values)

Model P Values: [4.64146108e-28 5.70145667e-01 2.85282740e-06]
Model Std Errs: [5.60533672 3.49727772 0.12458719]
Model Coef: [72.41965508  1.989221   -0.60060795]
